# 1. read csv and flatten json fields

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

def load_df(csv_path='train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir())

['.ipynb_checkpoints', 'GA_cx_revenue.ipynb', 'sample_submission.csv', 'test.csv', 'train-flattened.csv', 'train.csv']


In [2]:
%%time
df_small = load_df(nrows=10000)

Loaded train.csv. Shape: (10000, 54)
CPU times: user 2 s, sys: 103 ms, total: 2.1 s
Wall time: 2.21 s


In [3]:
%%time
df_train = load_df()
df_test = load_df("test.csv")

Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)
CPU times: user 9min 7s, sys: 19min 43s, total: 28min 51s
Wall time: 41min 7s


In [4]:
%%time
df_train.to_csv("train_flattened.csv", index=False)
df_test.to_csv("test_flattened.csv", index=False)

CPU times: user 1min 21s, sys: 24.8 s, total: 1min 46s
Wall time: 2min 21s
